In [1]:
import numpy as np

In [5]:
a = [
    [0.2, 0.1, 1.7],
    [-0.4, 0.5, -1.1],
    [0.3, 1.0, -0.6],
    [-0.1, 0.4]
]

In [6]:
for i in a:
    print(np.mean(i))

0.6666666666666666
-0.3333333333333333
0.23333333333333336
0.15000000000000002


In [ ]:
import autosklearn.regression
import sklearn.model_selection
import sklearn.metrics
import os
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFECV
import xgboost as xgb
from sklearn.model_selection import GridSearchCV


In [41]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')


In [4]:
df = []
src = '/media/palm/BiggerData/superai/baseline_ss3/final_iot/kaggle/ss3-final-iot-farm-connect/train'
for file in os.listdir(src):
    df.append(pd.read_csv(os.path.join(src, file)))
df = pd.concat(df)
df = df.fillna(0)
x_train = df[['1Air Humidity', '1AirTemperature', '1LightHigh',
                '1LightLow', '11Rain Accumulate (mm)', '11Rain count','12WindSpeed',
                '23SoilEC', '23SoilpH', '23SoilTemp.อุณหภูมิดิน',  '26N', '26P', '26K',
                '29SoilTension', 'flow1']]
y_train = df[['23SoilWC ค.ชื้นดิน']]

In [5]:
x_train.head()

,1Air Humidity,1AirTemperature,1LightHigh,1LightLow,11Rain Accumulate (mm),11Rain count,12WindSpeed,23SoilEC,23SoilpH,23SoilTemp.อุณหภูมิดิน,26N,26P,26K,29SoilTension,flow1
0,49.1,27.4,0.0,59.01,0.0,0.0,0.9,0.12,4.8,22.3,25.0,104.0,97.0,91.0,0.0
1,49.6,28.2,0.0,59.70,0.0,0.0,0.8,0.12,4.8,22.3,25.0,104.0,97.0,0.0,0.0
2,49.1,27.2,0.0,60.68,0.0,0.0,0.7,0.12,4.8,22.3,25.0,104.0,97.0,90.0,0.0
3,50.8,26.9,0.0,61.51,0.0,0.0,3.8,0.12,4.8,22.3,25.0,104.0,97.0,91.0,0.0
4,48.7,28.2,0.0,60.71,0.0,0.0,0.3,0.12,4.8,22.3,25.0,104.0,97.0,90.0,0.0


In [6]:
x_test = []
for f in ['02', '09']:
    x_test.append(
        pd.read_csv(
            f'/media/palm/BiggerData/superai/baseline_ss3/final_iot/kaggle/ss3-final-iot-farm-connect/test/set{f}.csv'
        )
    )
x_test = pd.concat(x_test)
x_test = x_test.fillna(0)
x_test = x_test[['1Air Humidity', '1AirTemperature', '1LightHigh',
                '1LightLow', '11Rain Accumulate (mm)', '11Rain count','12WindSpeed',
                '23SoilEC', '23SoilpH', '23SoilTemp.อุณหภูมิดิน',  '26N', '26P', '26K',
                '29SoilTension', 'flow1']]


In [7]:
x_test.head()

,1Air Humidity,1AirTemperature,1LightHigh,1LightLow,11Rain Accumulate (mm),11Rain count,12WindSpeed,23SoilEC,23SoilpH,23SoilTemp.อุณหภูมิดิน,26N,26P,26K,29SoilTension,flow1
0,46.2,30.7,65.53,8.79,0.0,0.0,1.3,0.11,7.9,22.4,21.0,95.0,88.0,26.0,0.0
1,46.0,31.4,65.53,8.26,0.0,0.0,0.0,0.11,7.9,22.4,21.0,95.0,88.0,26.0,0.0
2,44.4,32.2,65.53,8.66,0.0,0.0,1.0,0.11,7.9,22.3,21.0,95.0,88.0,0.0,0.0
3,43.0,33.0,65.53,8.68,0.0,0.0,0.0,0.11,7.9,22.4,21.0,95.0,88.0,26.0,0.0
4,42.4,32.9,65.53,8.86,0.0,0.0,0.0,0.11,7.9,22.4,21.0,95.0,88.0,26.0,0.0


# AutoML

In [ ]:
automl = autosklearn.regression.AutoSklearnRegressor()
automl.fit(x_train, y_train)
y_hat = automl.predict(x_test)


# RandomForrest

In [9]:
rf = RandomForestRegressor()
rf.fit(x_train, y_train)
y_hat = rf.predict(x_test)

/tmp/ipykernel_9001/2323612038.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(x_train, y_train)


# XGB

In [49]:
xb = xgb.XGBRegressor()
xb.fit(x_train, y_train)
y_hat = xb.predict(x_test)

## Feature selection

In [53]:
xb = xgb.XGBRegressor()
reg_mod = RFECV(xb,
                cv=3,
                step=5
                )

In [54]:
_ = reg_mod.fit(x_train, y_train)

In [55]:
selected_columns = x_train.columns[reg_mod.support_]
reg_mod.support_

array([False, False, False, False, False, False, False,  True,  True,
       False, False,  True,  True,  True, False])

In [56]:
train_data_selected = x_train[selected_columns]
test_data_selected = x_test[selected_columns]

In [58]:
xb = xgb.XGBRegressor()
xb.fit(train_data_selected, y_train)
y_hat = xb.predict(test_data_selected)

## Model Selection

In [76]:
param_grid = {
        "colsample_bytree": [0.3, 0.4, 0.5],
        "gamma": [0],
        "learning_rate": [0.4, 0.5, 0.6],
        "max_depth": [1, 2, 3],
        "reg_lambda": [11, 12, 13],
        "scale_pos_weight": [0],
        "subsample": [0.4, 0.5, 0.6],
}

In [77]:
grid_cv = xgb.XGBRegressor()
grid_cv = GridSearchCV(
    grid_cv, 
    param_grid, 
    n_jobs=-1, 
    cv=3, 
    scoring='neg_mean_absolute_error'
)
_ = grid_cv.fit(train_data_selected, y_train)
grid_cv.best_params_

{'colsample_bytree': 0.4,
 'gamma': 0,
 'learning_rate': 0.4,
 'max_depth': 3,
 'reg_lambda': 12,
 'scale_pos_weight': 0,
 'subsample': 0.5}

In [78]:
y_hat = grid_cv.predict(test_data_selected)

# submission

In [70]:
submission = pd.read_csv('/media/palm/BiggerData/superai/baseline_ss3/final_iot/kaggle/ss3-final-iot-farm-connect/sample_submission.csv')

In [71]:
submission['23SoilWC ค.ชื้นดิน'].iloc[1:] = y_hat[1:]

In [72]:
submission.head()

,index,23SoilWC ค.ชื้นดิน
0,02_1,31.100000
1,02_2,36.112343
2,02_3,35.275467
3,02_4,36.112343
4,02_5,36.112343


In [73]:
submission.to_csv('/media/palm/BiggerData/superai/baseline_ss3/final_iot/kaggle/ss3-final-iot-farm-connect/baseline.csv', index=False)

In [ ]:
df = pd.read_csv('soil.csv')
df.columns

Index(['1Air Humidity', '1AirTemperature', '1LightHigh', '1LightLow',
       '12WindSpeed', '13WindDirection', '23SoilWC ค.ชื้นดิน',
       '23SoilTemp.อุณหภูมิดิน', '23SoilEC', '23SoilpH', '26N', '26P', '26K',
       '29SoilTension', 'flow1'],
      dtype='object')


# Feature selection
df.corr().style.format("{:.4}").background_gradient(cmap=plt.get_cmap('coolwarm'), axis=1)

X = df[['23SoilEC','26P','26K','26N','1AirTemperature','23SoilTemp.อุณหภูมิดิน','23SoilpH','1LightLow']]     
y = df[['23SoilWC ค.ชื้นดิน']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.078, random_state = 99)

dt_param_grid = {
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 10, 20]
}
dt_grid_search = GridSearchCV(DecisionTreeRegressor(), dt_param_grid, cv=5, scoring='neg_mean_absolute_error')
dt_grid_search.fit(X_train, y_train)
dt_best = dt_grid_search.best_estimator_
dt_predictions = dt_best.predict(X_test)
dt_mae = mean_absolute_error(y_test, dt_predictions)
print(f'Decision Tree MSE: {dt_mae}, Best Params: {dt_grid_search.best_params_}')